# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Importing necessary Python packages 

In [15]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [16]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):              # to create a list of files and collect each filepath
    file_path_list = glob.glob(os.path.join(root,'*'))   # joining the file path and roots with the subdirectories using glob
#     print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [17]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

    
# creating a csv reader object to read the csv files, for every filepath in the file path list, 
# then extracting each data row one by one and appending it
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 

            
# printing the total number of rows at the end of the process
print(len(full_data_rows_list))


# creating a smaller event data csv file to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [18]:
# number of rows written on the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin Apache Cassandra Session 

#### Creating a Cluster and the session

In [19]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [20]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [21]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


# Query 1 - to get the artist name, song title, and song's lengh in a particular session and item, we need to create the table with session ID and itemInSession as the primary key that would make a more efficient query due to the unique indentification of the record.

# Creating a table with these columns - sessionId, itemInSession, artist, lenght, and song. The primary key would be composed of sessionId as the partition key and itemInSession as the clustering column because they are required to filter the rows. The other 3 columns are required in the question.

In [22]:
query = "CREATE TABLE IF NOT EXISTS event_data_new "
query = query + "(sessionId int, itemInSession int, artist text,  length double, song text, \
                    PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

To create an Appache Cassandra table from the CSV file, we need to read the data from CSV and assign them to appropriate columns

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)              # skip header
    
    for line in csvreader:
        query = "INSERT INTO event_data_new (sessionId, itemInSession, artist, length, song)"
        query = query + "VALUES (%s, %s,%s, %s, %s)"
        
        try:
            session.execute(query, (
                int(line[8]),  # sessionId (convert to integer)
                int(line[3]),  # itemInSession (convert to integer)
                line[0],  # artist
                float(line[5]),  # length (convert to float)
                line[9],  # song
            ))
            
        except Exception as e:
            print(e)

The query should use the event_data_new model to fetch the data required. To do that, we need to select sessionId, itemInSession, artist, length, song columns from the event_data_new table and filter the table by sessionId and itemInSession column. Since the 2 columns are already set as the primary key, it should easily fetch the required rows to answer the questions. 

In [25]:
query = "SELECT sessionId, itemInSession, artist, length, song \
        FROM event_data_new \
         WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length,)

Faithless Music Matters (Mark Knight Dub) 495.3073


# Query 2: Fetch only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
for userid = 10, sessionid = 182

The resulting table will have userId, sessionId, itemInSession, song, artist, firstName, and lastName columns. The composite primary key would be composed of userId and sessionId to group data by user and session and itemInSession as the clustering column. This will enable us to sort data by itemInSession as well as identifying rows by user and session.

Need to create a table named artist_song_user where userId (to uniquely identify each user and display their first and last names) and sessionId (to identify the session the user was in) would be set as the composite primary key and the clustering column would be itemInSession because we need to sort the resulting table by this column. 

In [30]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user "
query = query + "(userId int, sessionId int, itemInSession int, song text, artist text, \
                    firstName text, lastName text, \
                    PRIMARY KEY ((userId, sessionId), itemInSession))\
                    WITH CLUSTERING ORDER BY (itemInSession ASC);"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [31]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)              # skip header
    
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, song, artist, firstName, lastName)"
        query = query + "VALUES (%s, %s,%s, %s, %s, %s, %s)"
        
        try:
            session.execute(query, (
                int(line[10]),  # userId (convert to integer)
                int(line[8]),  # sessionId (convert to integer)
                int(line[3]),  # itemInSession (convert to integer)
                line[9],  # song
                line[0],  # artist
                line[1],  # firstName
                line[4],  # lastName
            ))
            
        except Exception as e:
            print(e)

Selecting the userId, sessionId, itemInSession, song, artist, firstName, lastName from the artist_song_user table to answer the questions. The table was modeled using userId and sessionId as the primary key to better run the query. 

In [32]:
query = "SELECT userId, sessionId, itemInSession, song, artist, firstName, lastName\
            FROM artist_song_user \
            WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Query 3: every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

As the filter needs to be set on the song and the results need to be presented by the user, we need a table with song, userId, firstName, and lastName columns and set the primary key of the table based on song and userId. 

In [34]:
query = "CREATE TABLE IF NOT EXISTS song_user "
query = query + "(song text, userId int, firstName text, lastName text, \
                    PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)              # skip header
    
    for line in csvreader:
        query = "INSERT INTO song_user (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s,%s, %s)"
        
        try:
            session.execute(query, (
                line[9],  # song
                int(line[10]),  # userId (convert to integer)
                line[1],  # firstName
                line[4],  # lastName
            ))
            
        except Exception as e:
            print(e)

Selecting song, userId, firstName, lastName columns from song_user table to display the first and last name of the users who listed to a particular song

In [36]:
query = "SELECT song, userId, firstName, lastName \
         FROM song_user \
         WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
#     print(row._fields)
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


# Dropping the tables created before closing the session

Dropping Table 1

In [18]:
query = "DROP table event_data_new"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Dropping Table 2

In [19]:
query = "DROP table artist_song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Dropping Table 3

In [20]:
query = "DROP table song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()